In [1]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib

In [2]:
pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

In [3]:
%matplotlib notebook

In [4]:
samples = pd.read_csv('samples.csv', header=None, names=['task', 'worker', 'label'])
samples

,task,worker,label
0,1,87,1
1,1,123,1
2,1,143,1
3,1,11,5
4,1,158,4
...,...,...,...
11670,2000,169,1
11671,2000,47,1
11672,2000,102,5
11673,2000,119,1


In [5]:
truth = pd.read_csv('truth-1.csv', header=None, names=['task', 'true_label'])
truth

,task,true_label
0,1,1
1,2,1
2,3,3
3,4,4
4,5,3
...,...,...
1886,1896,5
1887,1897,2
1888,1898,1
1889,1899,2


In [6]:
df = pd.merge(samples, truth, how='outer').set_index('worker')
df

,task,label,true_label
worker,,,
87,1,1,1.0
123,1,1,1.0
143,1,1,1.0
11,1,5,1.0
158,1,4,1.0
...,...,...,...
169,2000,1,NaN
47,2000,1,NaN
102,2000,5,NaN


In [7]:
X=df[df.true_label.notna()].iloc[:,:2].copy()
X

,task,label
worker,,
87,1,1
123,1,1
143,1,1
11,1,5
158,1,4
...,...,...
123,1900,4
83,1900,4
95,1900,3


In [8]:
X=X.reset_index().set_index(['worker','task']).unstack().fillna(0).values.T
print(X.shape)
X

(1891, 171)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 5.]])

In [9]:
Y=np.array(truth.true_label).reshape([1,1891]).T
Y.shape

(1891, 1)

In [10]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1512, 171)
(1512, 1)
(379, 171)
(379, 1)


In [11]:
model=GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1).fit(X_train, Y_train)
model.score(X_test, Y_test)

D:\ANACONDA2\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.941952506596306

In [12]:
# 保存模型
# joblib.dump(model, '963.pkl') 

In [13]:
#载入模型
# model_new=joblib.load('963.pkl') 